### Desafío 3: Elaboración de un modelo de clasificación
#### Etapa 2 - Balanceo de las clases

En la etapa de limpieza se detectó que la tabla no estaba balanceada, se procede a balancearla

#### 1. Lectura e interpretación de la tabla

In [98]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /opt/conda/envs/geopandas/lib/python3.7/site-packages (0.4.3)


In [99]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import confusion_matrix,roc_curve
from sklearn.metrics import precision_score, recall_score,accuracy_score
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE


In [100]:
#Leemos el dataset que se limpió previamente
df= pd.read_csv('Df_CyT_1.csv',low_memory=False)

In [101]:
#Se guarda el original por si se necesita posteriormente
df_original=df
df.head()

,Unnamed: 0,sexo,producciones_ult_anio,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,max_grado_academico,gran_area_experticia,area_experticia,disciplina_experticia,tipo_personal,condicion_docente,ded_horaria_docente,clase_cargo_docente,categoria_conicet,Presento_ultimo_año,rango_edad
0,1,F,1,3,2,2,Universitario de posgrado/doctorado,CIENCIAS NATURALES Y EXACTAS,Ciencias Físicas,Otras Ciencias Físicas,INVESTIGADOR,Interino,De 0 hasta 19 horas,Profesor titular,Investigador independiente,1,"(50, 60]"
1,2,M,9,3,4,4,Universitario de posgrado/doctorado,CIENCIAS MÉDICAS Y DE LA SALUD,Ciencias de la Salud,Enfermedades Infecciosas,INVESTIGADOR,Interino,De 20 hasta 39 horas,Jefe de trabajos prácticos,Investigador asistente,1,"(30, 40]"
2,3,M,0,0,0,6,Universitario de posgrado/doctorado,CIENCIAS MÉDICAS Y DE LA SALUD,Ciencias de la Salud,Parasitología,INVESTIGADOR,no docente,0,no docente,No pertenece a Conicet,0,"(60, 70]"
3,4,M,4,1,1,3,Universitario de grado,CIENCIAS NATURALES Y EXACTAS,Ciencias de la Tierra y relacionadas con el Me...,Geología,INVESTIGADOR,Regular o por concurso,De 0 hasta 19 horas,Ayudante de primera,No pertenece a Conicet,1,"(50, 60]"
4,5,F,0,0,0,1,Universitario de posgrado/maestría,CIENCIAS SOCIALES,Sociología,Otras Sociología,DOCENTE,Por contrato,De 0 hasta 19 horas,Profesor titular,No pertenece a Conicet,0,"(40, 50]"


In [102]:
#Se elimina la columna que no se usará
df.drop(columns='Unnamed: 0',inplace=True)

In [103]:
#Se analiza cuán desbalanceadas estás las clases
df['Presento_ultimo_año'].value_counts(normalize=True)

0    0.63118
1    0.36882
Name: Presento_ultimo_año, dtype: float64

In [104]:
df.columns

Index(['sexo', 'producciones_ult_anio', 'prod_año_anterior',
       'prod_2_años_anteriores', 'prod_3_años_anteriores',
       'max_grado_academico', 'gran_area_experticia', 'area_experticia',
       'disciplina_experticia', 'tipo_personal', 'condicion_docente',
       'ded_horaria_docente', 'clase_cargo_docente', 'categoria_conicet',
       'Presento_ultimo_año', 'rango_edad'],
      dtype='object')

El área de expertis de las personas se divide en 3 columnas: Gran área experticia, Área experticia, Disciplina experticias. Siendo la primera de todas la más abarcativa y la última la más detallada. Se probarán los modelos tomando las 3 opciones.
Para ellos, es necesario generar 3 tablas, separada cada una de ellas en Train y Test.

#### 2. Generación de dummies

In [105]:
#Antes de balancear se generan las dummies correspondientes 

In [106]:
#Se generan dummies para sexo 
df_dummies = pd.get_dummies(df['sexo'], prefix='sexo',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='sexo',inplace=True)

In [107]:
#Se generan dummies para rango_edad 
df_dummies = pd.get_dummies(df['rango_edad'], prefix='edad',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='rango_edad',inplace=True)

In [108]:
#Se generan dummies para tipo_personal
df_dummies = pd.get_dummies(df['tipo_personal'], prefix='t_personal',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='tipo_personal',inplace=True)

In [109]:
#Se generan dummies para condicion_docente
df_dummies = pd.get_dummies(df['condicion_docente'], prefix='cond_docente',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='condicion_docente',inplace=True)

In [110]:
#Se generan dummies para ded_horaria_docente
df_dummies = pd.get_dummies(df['ded_horaria_docente'], prefix='ded_horaria',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='ded_horaria_docente',inplace=True)

In [111]:
#Se generan dummies para max_grado_academico
df_dummies = pd.get_dummies(df['max_grado_academico'], prefix='grado_acad',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='max_grado_academico',inplace=True)

In [112]:
#Se generan dummies para categoria_conicet
df_dummies = pd.get_dummies(df['categoria_conicet'], prefix='cat_conicet',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='categoria_conicet',inplace=True)

In [113]:
#Se generan dummies para clase_cargo_docente
df_dummies = pd.get_dummies(df['clase_cargo_docente'], prefix='clase_cargo_docente',drop_first=True)
df=pd.concat([df,df_dummies], axis=1)
df.drop(columns='clase_cargo_docente',inplace=True)

#### 3. Generación de dummies y balanceo de la tabla para Gran área experticia

In [114]:
#Se generan dummies para 'gran_area_experticia'
df_dummies = pd.get_dummies(df['gran_area_experticia'], prefix='area_exp',drop_first=True)
df_gran_area=pd.concat([df,df_dummies], axis=1)
df_gran_area.drop(columns='gran_area_experticia',inplace=True)

In [115]:
#Se define la matriz target y la matriz de features
X_ga=df_gran_area.drop(columns=['producciones_ult_anio','area_experticia', 'disciplina_experticia',
       'Presento_ultimo_año'])

y_ga =df_gran_area['Presento_ultimo_año']

In [116]:
# Hacemos el split entre train y test
X_train_ga, X_test_ga, y_train_ga, y_test_ga = train_test_split(X_ga, y_ga, test_size = 0.33)

In [117]:
#Se analiza cuán desbalanceada está y_train
y_train_ga.value_counts(normalize=True)

0    0.628797
1    0.371203
Name: Presento_ultimo_año, dtype: float64

In [118]:
#Se analiza el tamaño de train
X_train_ga.shape,y_train_ga.shape

((43324, 100), (43324,))

In [119]:
#Se aplica SMOTE para oversamplear la base
sm = SMOTE(random_state=12)
X_train_ga_sm, y_train_ga_sm = sm.fit_sample(X_train_ga, y_train_ga)

In [120]:
X_train_ga_sm.shape, y_train_ga_sm.shape

((54484, 100), (54484,))

In [121]:
#X_train_ga_sm es un array, hay que pasarlo a DataFrame
X_train_ga_sm

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 2, 6, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 1, 4, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 3, 2, ..., 0, 0, 0]])

In [122]:
X_train_ga_sm=pd.DataFrame(X_train_ga_sm,columns=X_train_ga.columns)
X_train_ga_sm.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,clase_cargo_docente_Profesor por convenio,clase_cargo_docente_Profesor titular,clase_cargo_docente_Profesor titular plenario,clase_cargo_docente_Profesor visitante,clase_cargo_docente_no docente,area_exp_CIENCIAS MÉDICAS Y DE LA SALUD,area_exp_CIENCIAS NATURALES Y EXACTAS,area_exp_CIENCIAS SOCIALES,area_exp_HUMANIDADES,area_exp_INGENIERÍAS Y TECNOLOGÍAS
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,2,6,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,3,4,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [123]:
y_train_ga.value_counts(normalize=True)

0    0.628797
1    0.371203
Name: Presento_ultimo_año, dtype: float64

In [124]:
#Se observa que está balanceda
np.unique(y_train_ga_sm, return_counts=True)

(array([0, 1]), array([27242, 27242]))

In [128]:
#Se convierte y_train_ga_sm en DataFrame
y_train_ga_sm = pd.DataFrame(y_train_ga_sm)

In [96]:
y_train_ga_sm.head()

,0
0,0
1,0
2,0
3,1
4,1


In [132]:
#Se une X_train con y_train para guardar el archivo 
df_train_ga=X_train_ga_sm
df_train_ga['Presentó']=y_train_ga_sm
df_train_ga.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,clase_cargo_docente_Profesor titular,clase_cargo_docente_Profesor titular plenario,clase_cargo_docente_Profesor visitante,clase_cargo_docente_no docente,area_exp_CIENCIAS MÉDICAS Y DE LA SALUD,area_exp_CIENCIAS NATURALES Y EXACTAS,area_exp_CIENCIAS SOCIALES,area_exp_HUMANIDADES,area_exp_INGENIERÍAS Y TECNOLOGÍAS,Presentó
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,2,6,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,3,4,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [136]:
#Se una X_test con y_test
df_test_ga=X_test_ga
df_test_ga['Presentó']=y_test_ga
df_test_ga.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,clase_cargo_docente_Profesor titular,clase_cargo_docente_Profesor titular plenario,clase_cargo_docente_Profesor visitante,clase_cargo_docente_no docente,area_exp_CIENCIAS MÉDICAS Y DE LA SALUD,area_exp_CIENCIAS NATURALES Y EXACTAS,area_exp_CIENCIAS SOCIALES,area_exp_HUMANIDADES,area_exp_INGENIERÍAS Y TECNOLOGÍAS,Presentó
52873,3,8,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
45860,0,0,2,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
16362,0,0,5,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
36252,0,3,4,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
43437,4,3,2,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [137]:
#Se guardan los archivos para Gran Area Experticia 
df_train_ga.to_csv('df_train_ga.csv')
df_test_ga.to_csv('df_test_ga.csv')

#### 4. Generación de dummies y balanceo de la tabla para Área experticia

In [140]:
#Se generan dummies para 'area_experticia'
df_dummies = pd.get_dummies(df['area_experticia'], prefix='area_exp',drop_first=True)
df_area=pd.concat([df,df_dummies], axis=1)
df_area.drop(columns='area_experticia',inplace=True)

In [141]:
#Se define la matriz target y la matriz de features
X_a=df_area.drop(columns=['producciones_ult_anio','gran_area_experticia', 'disciplina_experticia',
       'Presento_ultimo_año'])

y_a =df_area['Presento_ultimo_año']

In [142]:
# Hacemos el split entre train y test
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.33)

In [143]:
#Se analiza cuán desbalanceada está y_train
y_train_a.value_counts(normalize=True)

0    0.630413
1    0.369587
Name: Presento_ultimo_año, dtype: float64

In [144]:
#Se aplica SMOTE para oversamplear la base
sm = SMOTE(random_state=12)
X_train_a_sm, y_train_a_sm = sm.fit_sample(X_train_a, y_train_a)

In [145]:
#Se convierte X_train_a_sm en DataFrame
X_train_a_sm=pd.DataFrame(X_train_a_sm,columns=X_train_a.columns)
X_train_a_sm.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,area_exp_Nanotecnología,area_exp_Otras Ciencias Agrícolas,area_exp_Otras Ciencias Médicas,area_exp_Otras Ciencias Naturales y Exactas,area_exp_Otras Ciencias Sociales,area_exp_Otras Humanidades,area_exp_Otras Ingenierías y Tecnologías,area_exp_Producción Animal y Lechería,area_exp_Psicología,area_exp_Sociología
0,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
#Se observa que está balanceda
np.unique(y_train_a_sm, return_counts=True)

(array([0, 1]), array([27312, 27312]))

In [147]:
#Se convierte y_train_a_sm en DataFrame
y_train_a_sm = pd.DataFrame(y_train_a_sm)
y_train_a_sm.head()

,0
0,0
1,0
2,0
3,0
4,0


In [148]:
#Se une X_train con y_train para guardar el archivo 
df_train_a=X_train_a_sm
df_train_a['Presentó']=y_train_a_sm
df_train_a.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,area_exp_Otras Ciencias Agrícolas,area_exp_Otras Ciencias Médicas,area_exp_Otras Ciencias Naturales y Exactas,area_exp_Otras Ciencias Sociales,area_exp_Otras Humanidades,area_exp_Otras Ingenierías y Tecnologías,area_exp_Producción Animal y Lechería,area_exp_Psicología,area_exp_Sociología,Presentó
0,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
#Se una X_test con y_test
df_test_a=X_test_a
df_test_a['Presentó']=y_test_a
df_test_a.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,area_exp_Otras Ciencias Agrícolas,area_exp_Otras Ciencias Médicas,area_exp_Otras Ciencias Naturales y Exactas,area_exp_Otras Ciencias Sociales,area_exp_Otras Humanidades,area_exp_Otras Ingenierías y Tecnologías,area_exp_Producción Animal y Lechería,area_exp_Psicología,area_exp_Sociología,Presentó
20850,0,1,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
38448,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
58343,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21497,0,2,2,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
48891,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
#Se guardan los archivos para Area Experticia 
df_train_a.to_csv('df_train_a.csv')
df_test_a.to_csv('df_test_a.csv')

#### 5. Generación de dummies y balanceo de la tabla para Disciplina experticia

In [151]:
#Se generan dummies para 'disciplina_experticia'
df_dummies = pd.get_dummies(df['disciplina_experticia'], prefix='area_exp',drop_first=True)
df_area=pd.concat([df,df_dummies], axis=1)
df_area.drop(columns='disciplina_experticia',inplace=True)

In [152]:
#Se define la matriz target y la matriz de features
X_disc=df_area.drop(columns=['producciones_ult_anio','gran_area_experticia', 'area_experticia',
       'Presento_ultimo_año'])

y_disc =df_area['Presento_ultimo_año']

In [153]:
# Hacemos el split entre train y test
X_train_disc, X_test_disc, y_train_disc, y_test_disc = train_test_split(X_disc, y_disc, test_size = 0.33)

In [154]:
#Se analiza cuán desbalanceada está y_train
y_train_disc.value_counts(normalize=True)

0    0.632028
1    0.367972
Name: Presento_ultimo_año, dtype: float64

In [156]:
#Se aplica SMOTE para oversamplear la base
sm = SMOTE(random_state=12)
X_train_disc_sm, y_train_disc_sm = sm.fit_sample(X_train_disc, y_train_disc)

In [157]:
#Se convierte X_train_a_sm en DataFrame
X_train_disc_sm=pd.DataFrame(X_train_disc_sm,columns=X_train_disc.columns)
X_train_disc_sm.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,"area_exp_Tópicos Sociales (estudios de género, estudios de familia, trabajo social, etc.)",area_exp_Urología y Nefrología,area_exp_Virología,area_exp_Vulcanología,"area_exp_Zoología, Ornitología, Entomología, Etología",area_exp_Ética (excepto ética relacionada con subáreas específicas),area_exp_Ética Médica,area_exp_Ética relacionada con Biotecnología Agrícola,area_exp_Ética relacionada con Biotecnología Médica,"area_exp_Óptica (incluida Óptica Láser y Óptica Cuántica), Acústica"
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,1,4,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
#Se observa que está balanceda
np.unique(y_train_disc_sm, return_counts=True)

(array([0, 1]), array([27382, 27382]))

In [159]:
#Se convierte y_train_disc_sm en DataFrame
y_train_disc_sm = pd.DataFrame(y_train_disc_sm)
y_train_disc_sm.head()

,0
0,0
1,0
2,0
3,1
4,1


In [160]:
#Se une X_train con y_train para guardar el archivo 
df_train_disc=X_train_disc_sm
df_train_disc['Presentó']=y_train_disc_sm
df_train_disc.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,area_exp_Urología y Nefrología,area_exp_Virología,area_exp_Vulcanología,"area_exp_Zoología, Ornitología, Entomología, Etología",area_exp_Ética (excepto ética relacionada con subáreas específicas),area_exp_Ética Médica,area_exp_Ética relacionada con Biotecnología Agrícola,area_exp_Ética relacionada con Biotecnología Médica,"area_exp_Óptica (incluida Óptica Láser y Óptica Cuántica), Acústica",Presentó
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,1,4,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [161]:
#Se una X_test con y_test
df_test_disc=X_test_disc
df_test_disc['Presentó']=y_test_disc
df_test_disc.head()

,prod_año_anterior,prod_2_años_anteriores,prod_3_años_anteriores,sexo_M,"edad_(20, 30]","edad_(30, 40]","edad_(40, 50]","edad_(50, 60]","edad_(60, 70]","edad_(70, 80]",...,area_exp_Urología y Nefrología,area_exp_Virología,area_exp_Vulcanología,"area_exp_Zoología, Ornitología, Entomología, Etología",area_exp_Ética (excepto ética relacionada con subáreas específicas),area_exp_Ética Médica,area_exp_Ética relacionada con Biotecnología Agrícola,area_exp_Ética relacionada con Biotecnología Médica,"area_exp_Óptica (incluida Óptica Láser y Óptica Cuántica), Acústica",Presentó
52486,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
24806,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
24957,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40228,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32766,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
#Se guardan los archivos para Disciplina Experticia 
df_train_disc.to_csv('df_train_disc.csv')
df_test_disc.to_csv('df_test_disc.csv')